<a href="https://www.kaggle.com/code/gopimali/pretrained-fine-tunned?scriptVersionId=131616147" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-task/Train_nlp.csv
/kaggle/input/nlp-model/GoogleNews-vectors-negative300.bin


In [2]:
path = '/kaggle/input/nlp-model/GoogleNews-vectors-negative300.bin'

In [3]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format(path,binary=True)

In [4]:
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
# from contractions import fix
from unidecode import unidecode
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [5]:
data = pd.read_csv('/kaggle/input/nlp-task/Train_nlp.csv')

In [6]:
model['good']

array([ 0.04052734,  0.0625    , -0.01745605,  0.07861328,  0.03271484,
       -0.01263428,  0.00964355,  0.12353516, -0.02148438,  0.15234375,
       -0.05834961, -0.10644531,  0.02124023,  0.13574219, -0.13183594,
        0.17675781,  0.27148438,  0.13769531, -0.17382812, -0.14160156,
       -0.03076172,  0.19628906, -0.03295898,  0.125     ,  0.25390625,
        0.12695312, -0.15234375,  0.03198242,  0.01135254, -0.01361084,
       -0.12890625,  0.01019287,  0.23925781, -0.08447266,  0.140625  ,
        0.13085938, -0.04516602,  0.06494141,  0.02539062,  0.05615234,
        0.24609375, -0.20507812,  0.23632812, -0.00860596, -0.02294922,
        0.05078125,  0.10644531, -0.03564453,  0.08740234, -0.05712891,
        0.08496094,  0.23535156, -0.10107422, -0.03564453, -0.04736328,
        0.04736328, -0.14550781, -0.10986328,  0.14746094, -0.23242188,
       -0.07275391,  0.19628906, -0.37890625, -0.07226562,  0.04833984,
        0.11914062,  0.06103516, -0.12109375, -0.27929688,  0.05

In [7]:
data.head(2)

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0


In [8]:
stopword_list = stopwords.words("english")
stopword_list.remove("no")
stopword_list.remove("nor")
stopword_list.remove("not")

def clean_text(data):
    tokens = word_tokenize(data)
    clean_data = [word.lower() for word in tokens if (word.lower() not in punctuation) and (word.lower() not in stopword_list) and ( len(word)>2) and (word.isalpha()) ]
    return clean_data

tokens = data.text.apply(clean_text)

In [9]:
data['tokens'] = tokens

In [10]:
data.head()

,text,label,tokens
0,I grew up (b. 1965) watching and loving the Th...,0,"[grew, watching, loving, thunderbirds, mates, ..."
1,"When I put this movie in my DVD player, and sa...",0,"[put, movie, dvd, player, sat, coke, chips, ex..."
2,Why do people who do not know what a particula...,0,"[people, not, know, particular, time, past, li..."
3,Even though I have great interest in Biblical ...,0,"[even, though, great, interest, biblical, movi..."
4,Im a die hard Dads Army fan and nothing will e...,1,"[die, hard, dads, army, fan, nothing, ever, ch..."


In [11]:
len(tokens)

40000

# Vector with pretrained

In [12]:
vector_with_pretrained = []
pretrained_key_error_words = []
for t in range(len(tokens)):
    vector_list = []
    for word in tokens[t]:
        try:
            vector = model[word]
            vector_list.append(vector)
        except KeyError:
            pretrained_key_error_words.append(word)
    final_token = np.mean(vector_list, axis = 0)
    vector_with_pretrained.append(final_token)
    

In [13]:
print(pretrained_key_error_words[0:10])

['virgil', 'jonatha', 'frakes', 'videoshop', 'isnt', 'didnt', 'moonlanding', 'cheque', 'frazer', 'mcnealy']


In [14]:
data['vector_with_pretrained'] = vector_with_pretrained

In [15]:
data.head()

,text,label,tokens,vector_with_pretrained
0,I grew up (b. 1965) watching and loving the Th...,0,"[grew, watching, loving, thunderbirds, mates, ...","[0.060559157, 0.039513286, 0.015592435, 0.1114..."
1,"When I put this movie in my DVD player, and sa...",0,"[put, movie, dvd, player, sat, coke, chips, ex...","[0.06501582, 0.008160873, -0.008187361, 0.1270..."
2,Why do people who do not know what a particula...,0,"[people, not, know, particular, time, past, li...","[0.059786648, 0.033370122, 0.01162423, 0.12186..."
3,Even though I have great interest in Biblical ...,0,"[even, though, great, interest, biblical, movi...","[0.080684885, 0.021379359, -0.02830236, 0.1350..."
4,Im a die hard Dads Army fan and nothing will e...,1,"[die, hard, dads, army, fan, nothing, ever, ch...","[0.034482926, 0.054618753, 0.008420254, 0.0965..."


# Fine tunned

In [16]:
from gensim.models import Word2Vec

In [17]:
final_text = tokens.to_list()

In [18]:
model = Word2Vec(final_text ,window=3,min_count=2)

In [19]:
model.wv['good']

array([-2.30928838e-01,  7.33316302e-01, -1.55431187e+00, -1.43013752e+00,
       -8.73385012e-01,  3.39738548e-01,  1.18396735e+00, -4.07215059e-01,
       -7.00197399e-01, -3.41827750e+00, -7.00783134e-01, -9.57078516e-01,
        4.70416486e-01, -6.74514771e-01,  8.73904347e-01, -8.61990154e-01,
        7.79170454e-01, -5.53384006e-01, -6.97595179e-01, -1.36483026e+00,
        3.87644410e-01,  3.53656054e-01,  4.26816374e-01, -9.45357800e-01,
        4.77248430e-02,  6.13945484e-01, -9.99113858e-01, -1.75082541e+00,
        1.21743453e+00,  3.88687104e-01,  1.10237867e-01, -1.50188351e+00,
        2.95285612e-01, -1.21919799e+00,  3.09852421e-01,  2.89566922e+00,
        1.68572640e+00,  1.70021608e-01,  7.57454753e-01, -1.36810255e+00,
       -7.33682066e-02, -1.48718581e-01,  2.30006412e-01,  6.40935898e-01,
       -4.23127025e-01, -2.86563098e-01,  1.60769448e-01,  9.60967481e-01,
       -4.08853352e-01, -1.21623802e+00,  1.26180676e-04, -2.30844378e+00,
        1.15533698e+00, -

In [20]:
vector_with_finetunned = []
fine_tunned_key_error_words = []
for t in range(len(tokens)):
    vector_list = []
    for word in tokens[t]:
        try:
            vector = model.wv[word]
            vector_list.append(vector)
        except KeyError :
            fine_tunned_key_error_words.append(word)
    final_token = np.mean(vector_list, axis = 0)
    vector_with_finetunned.append(final_token)

In [21]:
print(fine_tunned_key_error_words[0:10])

['jonatha', 'subsp', 'videoshop', 'sniggered', 'moonlanding', 'ipso', 'audiobooks', 'microsystem', 'okazaki', 'quaalude']


In [22]:
len(vector_with_finetunned)

40000

In [23]:
data['vector_with_finetunned'] = vector_with_finetunned

In [24]:
data.head()

,text,label,tokens,vector_with_pretrained,vector_with_finetunned
0,I grew up (b. 1965) watching and loving the Th...,0,"[grew, watching, loving, thunderbirds, mates, ...","[0.060559157, 0.039513286, 0.015592435, 0.1114...","[-0.42633098, 0.1820808, -0.18427855, 0.074281..."
1,"When I put this movie in my DVD player, and sa...",0,"[put, movie, dvd, player, sat, coke, chips, ex...","[0.06501582, 0.008160873, -0.008187361, 0.1270...","[-0.70374215, 0.24063809, -0.42858544, 0.17043..."
2,Why do people who do not know what a particula...,0,"[people, not, know, particular, time, past, li...","[0.059786648, 0.033370122, 0.01162423, 0.12186...","[-0.78967756, 0.34129566, -0.2184484, 0.192671..."
3,Even though I have great interest in Biblical ...,0,"[even, though, great, interest, biblical, movi...","[0.080684885, 0.021379359, -0.02830236, 0.1350...","[-0.86590004, 0.43821952, -0.19298178, 0.13204..."
4,Im a die hard Dads Army fan and nothing will e...,1,"[die, hard, dads, army, fan, nothing, ever, ch...","[0.034482926, 0.054618753, 0.008420254, 0.0965...","[-0.5856434, 0.25794038, -0.3560385, -0.022996..."
